In [0]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

class data_files(Dataset):
  def __init__(self, img_path, label_path, transforms=None, mode='train'):
    super(Dataset, self).__init__()
    self.img_path = img_path
    self.images = np.load(self.img_path)
    if mode == 'train':
      self.label_path = label_path
      self.labels = torch.from_numpy(np.load(self.label_path))

    self.transforms = transforms
    self.mode = mode

  def __len__(self):
    return self.images.shape[0]

  def __getitem__(self, index):
    img = self.images[index, ...]
    if self.transforms != None:
      img = self.transforms(img)
    
    if self.mode == 'train':
      label = self.labels[index]
      return img, label
    elif self.mode == 'test':
      return img
  

In [0]:
import torchvision.transforms as transforms
import torchvision

img_transforms = transforms.Compose([
                                    transforms.ToPILImage(),
                                    transforms.RandomRotation(30),
                                    transforms.ColorJitter(),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),                             
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                  ])

img_path = 'drive/My Drive/data/train-images.npy'
label_path = 'drive/My Drive/data/train-labels.npy'

In [0]:
dataset = data_files(img_path, label_path, transforms=img_transforms, mode='train') 

In [0]:
dataset.__len__()

43466

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

# needed more data for training thus reduced validation split
validation_split = .05
shuffle_dataset = True
random_seed= 42
BATCH = 512

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
trainsampler = SubsetRandomSampler(train_indices)
validsampler = SubsetRandomSampler(val_indices)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH, 
                                           sampler=trainsampler, num_workers=1)
validloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH,
                                                sampler=validsampler, num_workers=1)

In [0]:
model = torchvision.models.densenet201(pretrained=True, progress=True)
model.classifier = nn.Sequential(
                         nn.Linear(in_features=1920, out_features=2048),
                         nn.ReLU(True),
                         nn.Dropout(p=0.2),
                         nn.Linear(in_features=2048, out_features=38))
model = model.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [0]:
def train(trainloader, model, criterion, optimizer):

  model.train()

  losses = 0
  total = 0
  correct = 0

  for batch_idx, (inputs, targets) in enumerate(trainloader):
    inputs, targets = inputs.cuda(), targets.cuda()
    
    outputs = model(inputs)
    if batch_idx % 10 == 0:
      print(batch_idx, losses)
    
    loss =  criterion(outputs, targets)

    _, predicted = torch.max(outputs.data, 1)
    
    total += targets.size(0)
    correct += (predicted == targets).sum()
    losses += loss.item()*inputs.size(0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return (losses / total, 100.0 * correct / total)

In [0]:
def validate(validloader, model, criterion):
  model.eval()

  losses = 0
  total = 0
  correct = 0

  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(validloader):
      inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = model(inputs)
      loss =  criterion(outputs, targets)

      _, predicted = torch.max(outputs.data, 1)
      
      total += targets.size(0)
      correct += (predicted == targets).sum()
      losses += loss.item()*inputs.size(0)
    

  return (losses / total, 100.0 * correct / total)

In [0]:
NUM_EPOCHS = 4
def run_model(trainloader, validloader, model, criterion, optimizer):
  import time
  train_losses = []
  train_accuracy = []

  valid_losses = []
  valid_accuracy = []

  for epoch in range(NUM_EPOCHS):
    st = time.time()
    tlosses, tacc = train(trainloader, model, criterion, optimizer)
    # scheduler.step()/
    vlosses, vacc = validate(validloader, model, criterion)
    print('Epoch: {}:: train_loss: {}, train_accuracy: {} \n valid_loss: {}, valid_accuracy: {}'.format(
        epoch, tlosses, tacc,
        vlosses, vacc
    ))
    train_losses.append(tlosses)
    train_accuracy.append(tacc)
    valid_losses.append(vlosses)
    valid_accuracy.append(vacc)
    
    en = time.time()
    print("epoch took:", en - st)

  return {
      'train' : (train_losses, train_accuracy),
      'val'  : (valid_losses, valid_accuracy),
  }

result = run_model(trainloader, validloader, model, criterion, optimizer)

Use color jitter augmentations for some epochs and then remove it from the augmentations and train some more

In [0]:
torch.save(model.state_dict(), 'drive/My Drive/aicrowd_blitz/minileaves/model.pth')

In [0]:
model.eval()

testset = data_files('drive/My Drive/data/test-images.npy', None, transforms=img_transforms, mode='test')
testloader = torch.utils.data.DataLoader(testset, batch_size=512, 
                                           shuffle=False, num_workers=4)


In [0]:
model.eval()
preds = []
with torch.no_grad():
    for images in testloader:
        data = images.cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          preds.append(i)

In [0]:
import pandas as pd
pd.DataFrame(np.array(preds)).to_csv('drive/My Drive/data/densenet201-submission.csv', header=['class_index'])